In [32]:
import tkinter as tk
from tkinter import filedialog, messagebox
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Util.Padding import pad, unpad
import os

# Función para cifrar un archivo
def encrypt_file(file_path, output_path, key, salt):
    with open(file_path, 'rb') as file:
        file_data = file.read()
    iv = get_random_bytes(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    padded_data = pad(file_data, AES.block_size)
    ciphertext = cipher.encrypt(padded_data)
    with open(output_path, 'wb') as file_enc:
        file_enc.write(salt)  # Guardar la sal al inicio del archivo cifrado
        file_enc.write(iv)
        file_enc.write(ciphertext)

# Función para descifrar un archivo
def decrypt_file(file_path, output_path, key):
    with open(file_path, 'rb') as file_enc:
        salt = file_enc.read(16)  # Leer la sal
        iv = file_enc.read(16)  # Leer el IV
        ciphertext = file_enc.read()
    cipher = AES.new(key, AES.MODE_CBC, iv)
    padded_data = cipher.decrypt(ciphertext)
    try:
        file_data = unpad(padded_data, AES.block_size)
    except (ValueError, KeyError):
        messagebox.showerror("Error", "La clave es incorrecta o el archivo está dañado.")
        return
    with open(output_path, 'wb') as file:
        file.write(file_data)

# Función para seleccionar el archivo
def select_file():
    file_path = filedialog.askopenfilename(title="Selecciona el archivo")
    entry_file_path.delete(0, tk.END)
    entry_file_path.insert(0, file_path)

# Función para cifrar cuando se presiona el botón
def start_encryption():
    file_path = entry_file_path.get()
    password = entry_password.get()
    if not file_path or not password:
        messagebox.showwarning("Advertencia", "Por favor, selecciona un archivo e ingresa una contraseña.")
        return
    try:
        salt = get_random_bytes(16)  # Sal aleatoria para proteger la contraseña
        key = PBKDF2(password, salt, dkLen=32, count=1000000)
        file_name, file_extension = os.path.splitext(file_path)
        output_path = f"{file_name}.eas"
        encrypt_file(file_path, output_path, key, salt)
        messagebox.showinfo("Éxito", f"El archivo ha sido cifrado y guardado como {output_path}.")
    except Exception as e:
        messagebox.showerror("Error", f"Hubo un problema al cifrar el archivo: {str(e)}")

# Función para descifrar cuando se presiona el botón
def start_decryption():
    file_path = entry_file_path.get()
    password = entry_password.get()
    if not file_path or not password:
        messagebox.showwarning("Advertencia", "Por favor, selecciona un archivo e ingresa una contraseña.")
        return
    try:
        # Leer la sal del archivo cifrado para derivar la clave
        with open(file_path, 'rb') as file_enc:
            salt = file_enc.read(16)  # Leer la sal
        key = PBKDF2(password, salt, dkLen=32, count=1000000)
        file_name, file_extension = os.path.splitext(file_path)
        output_path = f"{file_name}_dec{file_extension}"
        decrypt_file(file_path, output_path, key)
        messagebox.showinfo("Éxito", f"El archivo ha sido descifrado y guardado como {output_path}.")
    except Exception as e:
        messagebox.showerror("Error", f"Hubo un problema al descifrar el archivo: {str(e)}")

# Crear la ventana principal
root = tk.Tk()
root.title("Programa para Cifrar y Descifrar")

# Etiqueta y campo para seleccionar el archivo
label_file_path = tk.Label(root, text="Archivo:")
label_file_path.grid(row=0, column=0, padx=10, pady=10)

entry_file_path = tk.Entry(root, width=50)
entry_file_path.grid(row=0, column=1, padx=10, pady=10)

button_browse = tk.Button(root, text="Elegir Archivo", command=select_file)
button_browse.grid(row=0, column=2, padx=10, pady=10)

# Etiqueta y campo para ingresar la contraseña
label_password = tk.Label(root, text="Contraseña:")
label_password.grid(row=1, column=0, padx=10, pady=10)

entry_password = tk.Entry(root, width=50, show="*")
entry_password.grid(row=1, column=1, padx=10, pady=10)

# Botón para iniciar el cifrado
button_encrypt = tk.Button(root, text="Cifrar", command=start_encryption)
button_encrypt.grid(row=2, column=0, pady=20)

# Botón para iniciar el descifrado
button_decrypt = tk.Button(root, text="Descifrar", command=start_decryption)
button_decrypt.grid(row=2, column=2, pady=20)

# Iniciar el loop principal de la aplicación
root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import os

# Función para cifrar un archivo con AES
def aes_encrypt_file(file_path, output_path, key):
    with open(file_path, 'rb') as file:
        file_data = file.read()
    cipher = AES.new(key, AES.MODE_CBC)
    padded_data = pad(file_data, AES.block_size)
    ciphertext = cipher.encrypt(padded_data)
    with open(output_path, 'wb') as file_enc:
        file_enc.write(cipher.iv)  # Guardar el IV al inicio del archivo cifrado
        file_enc.write(ciphertext)

# Función para descifrar un archivo con AES
def aes_decrypt_file(file_path, output_path, key):
    with open(file_path, 'rb') as file_enc:
        iv = file_enc.read(16)  # Leer el IV
        ciphertext = file_enc.read()
    cipher = AES.new(key, AES.MODE_CBC, iv)
    padded_data = cipher.decrypt(ciphertext)
    try:
        file_data = unpad(padded_data, AES.block_size)
    except (ValueError, KeyError):
        messagebox.showerror("Error", "La clave es incorrecta o el archivo está dañado.")
        return
    with open(output_path, 'wb') as file:
        file.write(file_data)

# Función para cifrar un archivo con RSA (solo para la clave AES)
def rsa_encrypt_key(aes_key, public_key):
    cipher = PKCS1_OAEP.new(public_key)
    return cipher.encrypt(aes_key)

# Función para descifrar la clave AES con RSA
def rsa_decrypt_key(encrypted_key, private_key):
    cipher = PKCS1_OAEP.new(private_key)
    return cipher.decrypt(encrypted_key)

# Función para seleccionar el archivo
def select_file():
    file_path = filedialog.askopenfilename(title="Selecciona el archivo")
    entry_file_path.delete(0, tk.END)
    entry_file_path.insert(0, file_path)

# Función para seleccionar el archivo de clave
def select_key_file(entry_key):
    key_file_path = filedialog.askopenfilename(title="Selecciona el archivo de clave")
    entry_key.delete(0, tk.END)
    entry_key.insert(0, key_file_path)

# Función para leer una clave desde un archivo
def read_key_from_file(file_path):
    with open(file_path, 'r') as key_file:
        key_data = key_file.read()
    return RSA.import_key(key_data.encode())

# Función para cifrar cuando se presiona el botón
def start_encryption():
    file_path = entry_file_path.get()
    public_key_file = entry_public_key.get()
    if not file_path or not public_key_file:
        messagebox.showwarning("Advertencia", "Por favor, selecciona un archivo y un archivo de clave pública.")
        return
    try:
        public_key = read_key_from_file(public_key_file)
        aes_key = get_random_bytes(32)  # Generar una clave AES de 256 bits
        encrypted_key = rsa_encrypt_key(aes_key, public_key)
        
        # Cifrar el archivo con AES
        file_name, file_extension = os.path.splitext(file_path)
        temp_output_path = f"{file_name}_temp.aes"
        aes_encrypt_file(file_path, temp_output_path, aes_key)

        # Guardar el archivo cifrado y la clave cifrada
        with open(f"{file_name}.rsa", 'wb') as file_enc:
            file_enc.write(encrypted_key)  # Guardar la clave AES cifrada
            with open(temp_output_path, 'rb') as temp_file:
                file_enc.write(temp_file.read())
        
        os.remove(temp_output_path)  # Eliminar el archivo temporal
        messagebox.showinfo("Éxito", f"El archivo ha sido cifrado y guardado como {file_name}.rsa.")
    except Exception as e:
        messagebox.showerror("Error", f"Hubo un problema al cifrar el archivo: {str(e)}")

# Función para descifrar cuando se presiona el botón
def start_decryption():
    file_path = entry_file_path.get()
    private_key_file = entry_private_key.get()
    if not file_path or not private_key_file:
        messagebox.showwarning("Advertencia", "Por favor, selecciona un archivo y un archivo de clave privada.")
        return
    try:
        private_key = read_key_from_file(private_key_file)
        with open(file_path, 'rb') as file_enc:
            encrypted_key = file_enc.read(256)  # Leer la clave AES cifrada
            aes_key = rsa_decrypt_key(encrypted_key, private_key)
            temp_output_path = f"{file_path}_temp.aes"
            with open(temp_output_path, 'wb') as temp_file:
                temp_file.write(file_enc.read())
        
        # Descifrar el archivo con AES
        file_name, file_extension = os.path.splitext(file_path)
        output_path = f"{file_name}_dec{file_extension}"
        aes_decrypt_file(temp_output_path, output_path, aes_key)

        os.remove(temp_output_path)  # Eliminar el archivo temporal
        messagebox.showinfo("Éxito", f"El archivo ha sido descifrado y guardado como {output_path}.")
    except Exception as e:
        messagebox.showerror("Error", f"Hubo un problema al descifrar el archivo: {str(e)}")

# Crear la ventana principal
root = tk.Tk()
root.title("Programa para Cifrar y Descifrar con RSA y AES")

# Etiqueta y campo para seleccionar el archivo
label_file_path = tk.Label(root, text="Archivo:")
label_file_path.grid(row=0, column=0, padx=10, pady=10)

entry_file_path = tk.Entry(root, width=50)
entry_file_path.grid(row=0, column=1, padx=10, pady=10)

button_browse = tk.Button(root, text="Elegir Archivo", command=select_file)
button_browse.grid(row=0, column=2, padx=10, pady=10)

# Etiqueta y campo para seleccionar el archivo de clave pública
label_public_key = tk.Label(root, text="Archivo Clave Pública:")
label_public_key.grid(row=1, column=0, padx=10, pady=10)

entry_public_key = tk.Entry(root, width=50)
entry_public_key.grid(row=1, column=1, padx=10, pady=10)

button_public_key = tk.Button(root, text="Elegir Clave Pública", command=lambda: select_key_file(entry_public_key))
button_public_key.grid(row=1, column=2, padx=10, pady=10)

# Etiqueta y campo para seleccionar el archivo de clave privada
label_private_key = tk.Label(root, text="Archivo Clave Privada:")
label_private_key.grid(row=2, column=0, padx=10, pady=10)

entry_private_key = tk.Entry(root, width=50)
entry_private_key.grid(row=2, column=1, padx=10, pady=10)

button_private_key = tk.Button(root, text="Elegir Clave Privada", command=lambda: select_key_file(entry_private_key))
button_private_key.grid(row=2, column=2, padx=10, pady=10)

# Botón para iniciar el cifrado
button_encrypt = tk.Button(root, text="Cifrar", command=start_encryption)
button_encrypt.grid(row=3, column=0, pady=20)

# Botón para iniciar el descifrado
button_decrypt = tk.Button(root, text="Descifrar", command=start_decryption)
button_decrypt.grid(row=3, column=2, pady=20)

# Iniciar el loop principal de la aplicación
root.mainloop()
